In [3]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch

sys.path.append(os.path.join(os.getenv("HOME"), "RNN_Manifold/"))
from manifold_encoder_decoder import s1_direct_product_generator, geometry_util, s1_direct_product_dimension_detecting_decoder

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
encoder, decoder = s1_direct_product_generator.train(1, 12, device, n_training_iterations=3000)
angles = np.arange(start=0, stop=2 * np.pi, step=0.01)
with torch.no_grad():
    points = geometry_util.torch_angles_to_ring(torch.tensor(angles, dtype=torch.get_default_dtype()).to(device))
    points = torch.unsqueeze(points, -2)
    ring_embedded_points = encoder(points)
ring_embedded_points = ring_embedded_points.cpu().numpy()


/home/trevor/brainvenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


iteration: 0, decoding loss: 1.0585721731185913, distance cost: 0.003723416943103075
iteration: 1, decoding loss: 0.9985806941986084, distance cost: 0.03511831536889076
iteration: 2, decoding loss: 0.4647623896598816, distance cost: 0.059866566210985184
iteration: 3, decoding loss: 0.3643225133419037, distance cost: 0.08677388727664948
iteration: 4, decoding loss: 0.3187597990036011, distance cost: 0.1276344507932663
iteration: 5, decoding loss: 0.2873629331588745, distance cost: 0.1411076784133911
iteration: 6, decoding loss: 0.26835545897483826, distance cost: 0.14995263516902924
iteration: 7, decoding loss: 0.28280067443847656, distance cost: 0.12105946987867355
iteration: 13, decoding loss: 0.3579474091529846, distance cost: 0.021010318771004677
iteration: 14, decoding loss: 0.28290295600891113, distance cost: 0.01615753024816513
iteration: 15, decoding loss: 0.25906649231910706, distance cost: 0.017814384773373604
iteration: 16, decoding loss: 0.25347036123275757, distance cost: 0

In [6]:
ring_embedded_points = ring_embedded_points/np.mean(np.abs(ring_embedded_points))

In [7]:
import importlib
importlib.reload(s1_direct_product_dimension_detecting_decoder)
encoder, decoder, geometry_profile = s1_direct_product_dimension_detecting_decoder.train(ring_embedded_points, 2, device, decoder_weight=10, scrambling_weight=1, order_red_weight=1, n_training_iterations=10000)


iteration: 0, decoding loss: 1.2936427593231201, distance cost: 0.017933841794729233, order reduction: 0.004556799307465553, scrambling weights: tensor([0.0180, 0.0180], device='cuda:0', grad_fn=<SigmoidBackward0>)
iteration: 1, decoding loss: 1.154798150062561, distance cost: 0.027477554976940155, order reduction: 0.012427660636603832, scrambling weights: tensor([0.0180, 0.0180], device='cuda:0', grad_fn=<SigmoidBackward0>)
iteration: 2, decoding loss: 0.5371384024620056, distance cost: 0.01715560257434845, order reduction: 0.002672301372513175, scrambling weights: tensor([0.0180, 0.0180], device='cuda:0', grad_fn=<SigmoidBackward0>)
iteration: 3, decoding loss: 0.18424092233181, distance cost: 0.00765997963026166, order reduction: 0.002349321497604251, scrambling weights: tensor([0.0180, 0.0180], device='cuda:0', grad_fn=<SigmoidBackward0>)
iteration: 6, decoding loss: 0.14293710887432098, distance cost: 0.02525051310658455, order reduction: 0.004527347162365913, scrambling weights: 

In [9]:
geometry_profile = geometry_profile.cpu().detach().numpy()

In [10]:
not_used_dimension = np.argmax(geometry_profile)

In [13]:
test_phases = np.linspace(start=-np.pi, stop=np.pi, num=100)
test_const_phase = np.ones_like(test_phases) * np.random.uniform(-np.pi, np.pi)
all_test_phases = np.zeros((test_phases.size, 2))
all_test_phases[:, not_used_dimension] = test_phases
all_test_phases[:, 1 - not_used_dimension] = test_const_phase
